In [ ]:
import numpy as np 
import pandas as pd 
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train_main = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv', index_col='id')
df_train = df_train_main.copy()
df_train.shape

In [ ]:
df_train.head(10)

In [ ]:
df_test_main = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv', index_col='id')
df_test = df_test_main.copy()
df_test.shape

In [ ]:
df_test.head(10)

In [ ]:
# train + test 
train_test = pd.concat([df_train.drop('target', axis=1), df_test])
train_test

In [ ]:
#테스트데이터 먼저 정의 
# test_temp = df_test_main.copy()
# test_temp.drop(['bin_0', 'bin_1', 'id'], axis='columns', inplace=True)

y_train = df_train['target'].copy()
y_train

In [ ]:
#na값이 있는지 삭제 
train_test = train_test.dropna()

In [ ]:
for column in train_test:
    print(column, " unique value length: ", len(train_test[column].unique()))

In [ ]:
train_test['bin_0'].value_counts().plot(kind='bar')

In [ ]:
#삭제 컬럼 
train_test.drop('bin_0', inplace=True, axis=1)

In [ ]:
train_test['bin_1'].value_counts().plot(kind='bar')

In [ ]:
# train_test = train_test.drop(['bin_1'], axis='columns')

In [ ]:
train_test['bin_3'].replace({"T" : 1, "F" : 0}, inplace=True)
train_test['bin_4'].replace({"Y" : 1, "N" : 0}, inplace=True)


### Label encoder 

In [ ]:
# for i in range(0,10):
#   column = f'nom_{i}'
#   le = preprocessing.LabelEncoder()
#   le.fit(train_test[column])
#   train_test[column] = le.transform(train_test[column])

In [ ]:
train_test

### Ordinary encoder

In [ ]:
train_test['ord_0'] = train_test['ord_0'] - 1

In [ ]:
train_test['ord_1'].replace({"Novice" : 0, "Contributor" : 1, "Expert" : 2, "Master" : 3, "Grandmaster" : 4}, inplace=True)
train_test['ord_2'].replace({"Freezing" : 5, "Cold" : 4, "Warm" : 3, "Hot" : 2, "Boiling Hot" : 1, "Lava Hot" : 0}, inplace=True)


In [ ]:
# ord_3, ord_4, ord_5 
oe = preprocessing.OrdinalEncoder(categories='auto')
train_test[['ord_3', 'ord_4', 'ord_5']] = oe.fit_transform(train_test[['ord_3', 'ord_4', 'ord_5']])
train_test[['ord_3', 'ord_4', 'ord_5']]

In [ ]:
oe.categories_

In [ ]:
# Scaling Ordianl cols 
ord_col = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
train_test[ord_col] = preprocessing.StandardScaler().fit_transform(train_test[ord_col])

In [ ]:
train_test

In [ ]:
#######
nom_col = ['nom_5', 'nom_6', 'nom_7','nom_8','nom_9']
train_test[nom_col]

In [ ]:

# train_test[nomvar[5:]].nunique()
train_test[nom_col].nunique()

# train_test['nom_5'] = train_test['nom_5'].astype('str').str[4:]
# train_test['nom_6'] = train_test['nom_6'].astype('str').str[3:]
# train_test['nom_7'] = train_test['nom_7'].astype('str').str[3:]
# train_test['nom_8'] = train_test['nom_8'].astype('str').str[3:]
# train_test['nom_9'] = train_test['nom_9'].astype('str').str[3:]

train_test['nom_5'] = train_test['nom_5'].str[4:]
train_test['nom_6'] = train_test['nom_6'].str[3:]
train_test['nom_7'] = train_test['nom_7'].str[3:]
train_test['nom_8'] = train_test['nom_8'].str[3:]
train_test['nom_9'] = train_test['nom_9'].str[3:]


In [ ]:
# train_test['nom_5'] = pd.to_numeric(train_test['nom_5'])
# train_test['nom_6'] = pd.to_numeric(train_test['nom_6'])
# train_test['nom_7'] = pd.to_numeric(train_test['nom_7'])
# train_test['nom_8'] = pd.to_numeric(train_test['nom_8'])
# train_test['nom_9'] = pd.to_numeric(train_test['nom_9'])
# train_test.fillna(0, inplace=True)

In [ ]:
train_test.columns

In [ ]:
#one hot 
nom_cols = ['nom_0', 'nom_1', 'nom_2', 'nom_3','nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
onehot = preprocessing.OneHotEncoder(categories='auto', dtype='float64', drop='first')
nom_matrix = onehot.fit_transform(train_test[nom_cols])
train_test.drop(nom_cols, inplace=True, axis=1)

In [ ]:
dm_cols = ['day', 'month']
onehot = preprocessing.OneHotEncoder(categories='auto', dtype='float64', drop='first')
dm_matrix = onehot.fit_transform(train_test[dm_cols])
train_test.drop(dm_cols, inplace=True, axis=1)

In [ ]:
# dm_matrix
import scipy 

df_matrix = scipy.sparse.hstack([nom_matrix, scipy.sparse.coo_matrix(train_test).astype('float64'),
                                dm_matrix]).tocsr() 
display(df_matrix)

In [ ]:
X_train = df_matrix[:300000]
X_test = df_matrix[300000:]

#cross val  적용해보기  ##############################################

In [ ]:
y_train

In [ ]:
logreg = LogisticRegression( max_iter=1000, solver='lbfgs', n_jobs=-1)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
y_pred.shape

In [ ]:
lg=LogisticRegression(C=0.1245, solver='saga',penalty='l2', max_iter=1000, verbose=0, n_jobs=-1,class_weight="balansed",random_state=42)
lg.fit(X_train,y_train)
lg.score(X_train, y_train)

In [ ]:

C = 0.125
logreg2 = LogisticRegression(C=C, class_weight={0:1, 1:1.023608741232618}, max_iter=1000, solver='lbfgs', n_jobs=-1)
logreg2.fit(X_train, y_train)
logreg2.score(X_train, y_train)

from sklearn.metrics import roc_auc_score 
roc_auc_score(y_train, logreg2.predict(X_train))

In [ ]:
# !pip install lightgbm
from lightgbm import LGBMClassifier

model = LGBMClassifier(n_estimators=500)
model.fit(X_train, y_train)
model.score(X_train, y_train)   #, model.score(X_test, y_test)

In [ ]:
# prediction
y_pred = lg.predict(X_test) 
# y_pred = logreg.predict(X_test) 
# y_pred = model.predict(X_test) 
submission = pd.read_csv('/kaggle/input/cat-in-the-dat/sample_submission.csv', index_col='id')
submission['target'] = y_pred
submission.to_csv('submission.csv')


In [ ]:
!ls

In [ ]:
# df_test_main.to_csv('df_test_main_20220626.csv', header=None)
!kaggle competitions submit -c cat-in-the-dat -f submission.csv -m "Message"

In [ ]:
submission

In [ ]:
# df_test_main.shape
# df_test_main.columns

In [ ]:
# features.shape
# features.columns

In [ ]:
# set(df_test_main.columns)  - set(features.columns)

In [ ]:
# cross val 적용해보기/ baysican optimization 적용 
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold
import optuna 

kfold = StratifiedKFold(5)

#로지스틱에서 최적 parameter 찾기 
def objective(trial):
    clasas_weight=trial.suggest_uniform('clasas_weight', 1, 2)
    model=LogisticRegression(C=0.123456789, class_weight={0:1, 1:clasas_weight},max_iter=10000, solver='lbfgs', n_jobs=-1)
    score=-cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc').mean()
    return score
study=optuna.create_study()
study.optimize(objective, n_trials=50)

In [ ]:
print(study.best_params)
print(-study.best_value)
params=study.best_params

In [ ]:
C = 0.123456789

# clf = LogisticRegression(C=C, solver='lbfgs', max_iter=1000, class_weight={0:1, 1:1.0391707400969836}, verbose=0, n_jobs=-1)
clf = LogisticRegression(C=C, solver='lbfgs', max_iter=1000, class_weight={0:1, 1:1.02850768675501032}, verbose=0, n_jobs=-1)


score = cross_validate(clf, X_train, y_train, cv=3, scoring="roc_auc")
mean = score['test_score'].mean()
print(score['test_score'])
print('C =', C, f'{mean:.8f}')

In [ ]:
clf = LogisticRegression(C=C, solver='lbfgs',class_weight={0:1, 1:1.02850768675501032}, max_iter=1000, verbose=0, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
y_preds = clf.predict_proba(X_test)[:,1]

In [ ]:
result = pd.read_csv('/kaggle/input/cat-in-the-dat/sample_submission.csv', index_col='id')
result['target'] = y_preds
result.to_csv('submission.csv')

In [ ]:
!pip install kaggle

In [ ]:
!kaggle competitions submit -c cat-in-the-dat -f /kaggle/working/submission.csv -m "Message"

